<a href="https://colab.research.google.com/github/MarshaGomez/DNN-Sketches-image-analysis/blob/main/Code/sketches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting up the project


In [ ]:
import numpy as np
from numpy.linalg import norm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
zip_path = '/content/gdrive/Shareddrives/COMPUTER_VISION/MIM_zipped.zip'
!cp "{zip_path}" .
!unzip -q MIM_zipped.zip
!rm MIM_zipped.zip


### create Test (utile per quando faremo il training)

In [ ]:
import os 
from IPython.display import display
from ipywidgets import Image

#data loading
# reading from unzipped
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
# FILELIST_PATH = BASE_DIR + "/filelist.txt"
SKETCHES_DIR = BASE_DIR + "/sketches"
MIRFLICKR_DIR = BASE_DIR + "/mirflickr/mirflickr25k"

#print(FILELIST_PATH)

#num_folders = os.listdir(SKETCHES_DIR)
#print(num_folders)

In [ ]:
import os, shutil
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/all')

In [ ]:
test_images_per_class = 20 # le immagini sono al massimo 80
try:
  os.rename(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/train')
except:
  print('already renamed')

if not os.path.exists(f'{SKETCHES_DIR}/test'):
  os.mkdir(f'{SKETCHES_DIR}/test')
  for class_folder in os.listdir(f'{SKETCHES_DIR}/train'):
    if os.path.isdir(f'{SKETCHES_DIR}/train/{class_folder}'):
      images = os.listdir(f'{SKETCHES_DIR}/train/{class_folder}')
      os.mkdir(f'{SKETCHES_DIR}/test/{class_folder}')
      for i in range(test_images_per_class):
        os.rename(f'{SKETCHES_DIR}/train/{class_folder}/{images[i]}', f'{SKETCHES_DIR}/test/{class_folder}/{images[i]}')


### LSH Class


In [ ]:
# def singleton(class_):
#     instances = {}
#     def getinstance(*args, **kwargs):
#         if class_ not in instances:
#             instances[class_] = class_(*args, **kwargs)
#         return instances[class_]
#     return getinstance

# @singleton
# per adesso è più scomodo che utile averlo come singleton
import time

class LSH:
  def __init__(self, feature_dim, g = 10, h = 20, w = 4, bitwise_hash = False):
    """
    We have to find a way to load the stored index if exists or initialize the 
    initial structure
    """
    self._index = {}
    if bitwise_hash:
      # self.x = np.random.randn(g, h, feature_dim) # non sembra richieda una distribuzione normale
      self.x = np.random.normal(size=(g, h, feature_dim))
    else:
      self.x = np.random.normal(size=(g, h, feature_dim))
    self.w = np.ones((g, h, 1)) * w
    self.b = np.random.rand(g, h, 1) * w
    self.bitwise_hash = bitwise_hash # another way to create h (h will be only 0 o 1)

  def _hash(self, features):
    """
    crea l'hash di più cose contemporaeamente si aspetta un array composto dalle features una sotta l'altra (linguaggio super matematico)
    """
    # g = np.trunc((np.dot(p, self.x) + self.b) / self.w)
    #g = np.trunc((np.dot(self.x, p) + self.b) / self.w) questo funziona con 1
    # g = np.transpose(np.trunc(((np.dot(self.x, p.T) + self.b) / self.w)), (0,2,1)) miglior modo di vederlo
    if self.bitwise_hash:
      return (np.transpose(np.dot(self.x, features.T), (0,2,1)) > 0).astype(int).astype(str)
    return np.transpose(np.trunc(((np.dot(self.x, features.T) + self.b) / self.w)), (0,2,1)).astype(int).astype(str)

  def insert(self, features, ids, labels):
    """
    Insert new data, ci aspettiamo un array d
    """
    g = self._hash(features)
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    
    number_elements = len(ids)
    i = 0
    # print("hash calculated")
    # print(g.shape)
    g_index = -1
    for g_function in g:
      start_inner_for = time.time()
      g_index += 1
      for row in g_function:
        if i % 10000 == 0:
          start = time.time()
        
        bucket_id = str(g_index) + '_' + ','.join(row)
        
        if i % 10000 == 0:
            end = time.time()
            # print(f'join {end - start}')
        if not bucket_id in self._index:
          # self._index[bucket_id] = { 'features': np.array([features[i % number_elements]]), 'ids': np.array([ids[i % number_elements]]), 'labels': np.array([labels[i % number_elements]])}
          self._index[bucket_id] = { 'features': [features[i % number_elements]], 'ids': [ids[i % number_elements]], 'labels': [labels[i % number_elements]]}
          if i % 10000 == 0:
            end = time.time()
            # print(f'not in bucket {end - start}')
        else:
          if ids[i % number_elements] in self._index[bucket_id]['ids']:
            # print("duplicate")
            continue
          if i % 10000 == 0:
            end = time.time()
            # print(f'checking duplicates {end - start}')
          # print("collision inserted")
          # self._index[bucket_id]['features'] = np.vstack((self._index[bucket_id]['features'], features[i % number_elements]))
          self._index[bucket_id]['features'].append(features[i % number_elements])
          # self._index[bucket_id]['ids'] = np.vstack((self._index[bucket_id]['ids'], ids[i % number_elements]))
          self._index[bucket_id]['ids'].append(ids[i % number_elements])
          # self._index[bucket_id]['labels'] = np.vstack((self._index[bucket_id]['labels'], labels[i % number_elements]))
          self._index[bucket_id]['labels'].append(labels[i % number_elements])
          if i % 10000 == 0:
            end = time.time()
            # print(f'stacking {end - start}')
        i += 1
        assert i > 0, 'out of bound'
      end_inner_for = time.time()
      # print(f'inner for time: {end_inner_for - start_inner_for}')
    
    for bucket_id in self._index:
      self._index[bucket_id]['features'] = np.array(self._index[bucket_id]['features'])
      # print(self._index[bucket_id]['features'].shape)
      self._index[bucket_id]['ids'] = np.array(self._index[bucket_id]['ids'], )
      self._index[bucket_id]['labels'] = np.array(self._index[bucket_id]['labels'])
      

  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    """
    Query the data
    """
    g = self._hash(np.array([features]))
    i = 0
    k = None
    top_k += 1 # per far ritornare k e non k - 1
    cost = 0
    g_index = -1
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    for g_function in g:
      g_index += 1
      for row in g_function:
        bucket_id = str(g_index) + '_' + ','.join(row)
        # print(bucket_id)
        if bucket_id in self._index:
          # posso avere duplicati perchè se i punti vengono inseriti in più bucket, posso avere duplicati
          # quindi devo eliminarli
          # l'ho messo qua fuori che il controllo duplicati è uguale per tutte e due le distanze
          # print(f'bucket {bucket_id}')
          if k is not None:
            # print("duplicate")
            duplicate_index = np.isin(self._index[bucket_id]['ids'], k['ids'])
            if duplicate_index.all():
              continue; # se sono tutti duplicati non ha senso contare nulla
            bucket = {}
            bucket['ids'] = self._index[bucket_id]['ids'][~duplicate_index] # prendo quelli che non sono duplicati
            # print(duplicate_index)
            # print(self._index[bucket_id]['features'])
            bucket['features'] = self._index[bucket_id]['features'][~duplicate_index.flatten()] # each duplicate index must delete a row of features
            bucket['labels'] = self._index[bucket_id]['labels'][~duplicate_index]
          else:
            bucket = self._index[bucket_id]
        
          if mode == 'euclidean':
            # print(bucket['features'].shape)
            dist = norm(bucket['features'] - np.array(features), axis=1)
            # print(f'dist shape {dist.shape} and dist size {dist.size}')
            cost += dist.size
            if k is None:
              idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
              if dist.shape[0] - 1 > top_k - 1:  
                  idx_partitioned = idx_partitioned[:top_k - 1]
              k = {}
              # qua è più comodo avere array 1- dimensionali
              k['ids'] = bucket['ids'][idx_partitioned].flatten()
              k['labels'] = bucket['labels'][idx_partitioned].flatten()
              k['distances'] = dist[idx_partitioned]
              continue
            # https://stackoverflow.com/questions/10337533/a-fast-way-to-find-the-largest-n-elements-in-an-numpy-array
            # argpartition sembra essere incredibilmente veloce
            # ma non ordina completamente, ordina solo rispetto un punto, nel senso
            # io gli sto dicendo butta quelli più piccoli di k da una parte e quelli più grandi all'altra, ma non sto ordinando
            if k['distances'].shape[0] < top_k:
                # print((k['distances'].shape, dist.shape))
                distances = np.concatenate((k['distances'], dist))
                # print((k['ids'].shape, bucket['ids'].shape))
                # print(k['ids'])
                # print(bucket['ids'])
                ids = np.concatenate((k['ids'], bucket['ids']))
                # print((k['labels'].shape, bucket['labels'].shape))
                labels = np.concatenate((k['labels'], bucket['labels']))
                idx_sorted = np.argpartition(distances, top_k - 1 if distances.shape[0] - 1 > top_k else distances.shape[0] - 1)
                if distances.shape[0] - 1 > top_k - 1:  
                  idx_sorted = idx_sorted[:top_k - 1]
                k['ids'] = ids[idx_sorted]
                k['labels'] = labels[idx_sorted]
                k['distances'] = distances[idx_sorted]
                # print(f'k = {k}')
                continue

            idx = dist < np.max(k['distances'])
            # print(f"idx = {idx}")
            if np.any(idx):
              distances = np.concatenate((k['distances'], dist[idx]))
              ids = np.concatenate((k['ids'], bucket['ids'][idx]))
              labels = np.concatenate((k['labels'], bucket['labels'][idx]))
              idx_sorted = np.argpartition(distances, top_k - 1 if distances.shape[0] - 1 > top_k else distances.shape[0] - 1)
              if distances.shape[0] - 1 > top_k - 1:  
                  idx_sorted = idx_sorted[:top_k - 1]
              k['ids'] = ids[idx_sorted]
              k['labels'] = labels[idx_sorted]
              k['distances'] = distances[idx_sorted]

          else:
            # print(bucket['features'].shape)
            sim = np.sum(bucket['features'] * np.array(features), axis=1) / (norm(bucket['features'], axis=1) * norm(np.array([features]), axis=1))
            # print(f'sim shape {sim.shape} and sim size {sim.size}')
            cost += sim.size
            if k is None:
              idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
              if sim.shape[0] - 1 > top_k - 1:  
                idx_partitioned = idx_partitioned[-(top_k - 1):]
              k = {}
              # qua è più comodo avere array 1- dimensionali
              k['ids'] = bucket['ids'][idx_partitioned].flatten()
              k['labels'] = bucket['labels'][idx_partitioned].flatten()
              k['similarities'] = sim[idx_partitioned]
              continue
            # https://stackoverflow.com/questions/10337533/a-fast-way-to-find-the-largest-n-elements-in-an-numpy-array
            # argpartition sembra essere incredibilmente veloce
            # ma non ordina completamente, ordina solo rispetto un punto, nel senso
            # io gli sto dicendo butta quelli più piccoli di k da una parte e quelli più grandi all'altra, ma non sto ordinando
            if k['similarities'].shape[0] < top_k:
                # print((k['similarities'].shape, sim.shape))
                similarities = np.concatenate((k['similarities'], sim))
                # print((k['ids'].shape, bucket['ids'].shape))
                # print(k['ids'])
                # print(bucket['ids'])
                ids = np.concatenate((k['ids'], bucket['ids']))
                # print((k['labels'].shape, bucket['labels'].shape))
                labels = np.concatenate((k['labels'], bucket['labels']))
                idx_sorted = np.argpartition(similarities, -(top_k - 1) if similarities.shape[0] - 1 > top_k - 1 else similarities.shape[0] - 1)
                if similarities.shape[0] - 1 > top_k - 1:  
                  idx_sorted = idx_sorted[-(top_k - 1):]
                k['ids'] = ids[idx_sorted]
                k['labels'] = labels[idx_sorted]
                k['similarities'] = similarities[idx_sorted]
                # print(f'k = {k}')
                continue

            idx = sim > np.min(k['similarities'])
            # print(f"idx = {idx}")
            if np.any(idx):
              similarities = np.concatenate((k['similarities'], sim[idx]))
              ids = np.concatenate((k['ids'], bucket['ids'][idx]))
              labels = np.concatenate((k['labels'], bucket['labels'][idx]))
              idx_sorted = np.argpartition(similarities, -(top_k - 1) if similarities.shape[0] - 1 > top_k - 1 else similarities.shape[0] - 1)
              if similarities.shape[0] - 1 > top_k - 1:  
                idx_sorted = idx_sorted[-(top_k - 1):]
              k['ids'] = ids[idx_sorted]
              k['labels'] = labels[idx_sorted]
              k['similarities'] = similarities[idx_sorted]
        i += 1
    # ora ordino totalmente i risultati
    if k is None:
      return {} #zero result
    if mode == 'euclidean':
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted[:top_k - 1 if k['distances'].shape[0] - 1 > top_k else k['distances'].shape[0]]
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, cost)
      return k
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted[:top_k - 1 if k['similarities'].shape[0] - 1 > top_k else k['similarities'].shape[0]]
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, cost)
    return k

  def store(self):
    pass


In [ ]:
a = np.array([4, 3, 2, 1, 0])
a[np.argpartition(a,0)]

array([0, 3, 2, 1, 4])

In [ ]:
class NO_INDEX:
  def __init__(self):
    self.features = None
    self.ids = None
    self.labels = None

  def insert(self, features, ids, labels):
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    self.features = features
    self.ids = ids
    self.labels = labels
  
  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    top_k += 1
    if mode == 'euclidean':
      dist = norm(self.features - np.array(features), axis=1)
      idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
      k = {}
      # qua è più comodo avere array 1- dimensionali
      k['ids'] = self.ids[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['labels'] = self.labels[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['distances'] = dist[idx_partitioned][:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, dist.size)
      return k
    sim = np.sum(self.features * np.array(features), axis=1) / (norm(self.features, axis=1) * norm(np.array([features]), axis=1))
    idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
    k = {}
    # qua è più comodo avere array 1- dimensionali
    if sim.shape[0] - 1 > top_k - 1:  
      idx_partitioned = idx_partitioned[-(top_k - 1):]

    k['ids'] = self.ids[idx_partitioned].flatten()
    k['labels'] = self.labels[idx_partitioned].flatten()
    k['similarities'] = sim[idx_partitioned]
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, sim.size)
    return k
    

In [ ]:
#### TESTING PURPOSE
print('"Base LSH"')
index = LSH(feature_dim= 2, g= 2, h=2)
print('x=')
print(index.x)
print('w=')
print(index.w)
print('b=')
print(index.b)
print('ps =')
print(np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]))
print('insert')
index.insert(
     np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]), 
     np.array(['uno', 'due', 'tre','quattro']),
     np.array([0,0,1,1])
     )
print(index._index)
print('query euclidean')
print(index.query([1,1], 4, return_cost=True))
print('query similarity')
print(index.query([1,1], 4, mode='similarity', return_cost=True))
print(index)


print('"Differentiate g LSH"')
index = LSH(feature_dim= 2, g= 2, h=2)
print('x=')
print(index.x)
print('w=')
print(index.w)
print('b=')
print(index.b)
print('ps =')
print(np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]))
print('insert')
index.insert(
     np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]), 
     np.array(['uno', 'due', 'tre','quattro']),
     np.array([0,0,1,1])
     )
print(index._index)
print('query')
print(index.query([1,1], 4))
print(index)


print('"bitwise hash LSH"')
index = LSH(feature_dim= 2, g=2, h=2, bitwise_hash=True)
print('x=')
print(index.x)
print('w=')
print(index.w)
print('b=')
print(index.b)
print('ps =')
print(np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]))
print('insert')
index.insert(
     np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]), 
     np.array(['uno', 'due', 'tre','quattro']),
     np.array([0,0,1,1])
     )
print(index._index)
print('query')
print(index.query([1,1], 4))
print(index)

print('NO INDEX')
index = NO_INDEX()
print(np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]))
print('insert')
index.insert(
     np.array([[1,1], [1.1, 1.2], [1.4, 1.44], [3, 4]]), 
     np.array(['uno', 'due', 'tre','quattro']),
     np.array([0,0,1,1])
     )
print('query euclidean')
print(index.query([1,1], 4, return_cost=True))
print('query similarity')
print(index.query([1,1], 4, mode='similarity', return_cost=True))
print(index)


"Base LSH"
x=
[[[ 0.52305329  1.9471501 ]
  [-0.47508692  1.15829659]]

 [[ 0.3993382   0.93030108]
  [ 0.43676412  1.49028361]]]
w=
[[[4.]
  [4.]]

 [[4.]
  [4.]]]
b=
[[[1.73258229]
  [0.88037734]]

 [[2.1518298 ]
  [0.6339392 ]]]
ps =
[[1.   1.  ]
 [1.1  1.2 ]
 [1.4  1.44]
 [3.   4.  ]]
insert
{'0_1,0': {'features': array([[1.  , 1.  ],
       [1.1 , 1.2 ],
       [1.4 , 1.44]]), 'ids': array(['uno', 'due', 'tre'], dtype='<U3'), 'labels': array([0, 0, 1])}, '0_2,1': {'features': array([[3., 4.]]), 'ids': array(['quattro'], dtype='<U7'), 'labels': array([1])}, '1_0,0': {'features': array([[1. , 1. ],
       [1.1, 1.2]]), 'ids': array(['uno', 'due'], dtype='<U3'), 'labels': array([0, 0])}, '1_1,0': {'features': array([[1.4 , 1.44]]), 'ids': array(['tre'], dtype='<U3'), 'labels': array([1])}, '1_1,1': {'features': array([[3., 4.]]), 'ids': array(['quattro'], dtype='<U7'), 'labels': array([1])}}
query euclidean
({'ids': array(['uno', 'due', 'tre'], dtype='<U3'), 'labels': array([0, 0, 1]

### Loading images DATA


In [ ]:
# sketch image displaying 
filename = "airplane/1.png"
image1 = os.path.join(SKETCHES_DIR + '/all', filename)
display(Image.from_file(image1, width=229, height=229))

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04W\x00\x00\x04W\x08\x00\x00\x00\x00\x105\xd1!\x00\…

In [ ]:
# mirflickr image displaying 
filename = "png/im20.jpg"
image1 = os.path.join(MIRFLICKR_DIR, filename)
display(Image.from_file(image1, width=229, height=229))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import itertools
#yields batches of images from the subdirectories 
#found in the sketches directory [class_0 ... class_250]
#together with class labels.

#images normalization 
sketches_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
mirflickr_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/all',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

mirflickr_generator = mirflickr_datagen.flow_from_directory(
        # This is the target directory
        MIRFLICKR_DIR,
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)


# unique, counts = np.unique(sketches_generator.classes, return_counts=True)
# labels_dict = dict(zip(unique, counts))
# print(labels_dict)

Found 20000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.


### Loading DNN for features extraction

In [ ]:
#loading Inception DNN
from tensorflow.keras.applications.inception_v3 import InceptionV3
# provare con la v2
conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=INPUT_SHAPE,
                  pooling='avg')

conv_base.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________

In [ ]:
#Extracting features using the pretrained convolutional base 

def extract_features(extractor, generator, sample_count, dim=2048):
  features = np.zeros((sample_count, dim)) #extractor output shape 
  i = 0
  for inputs_batch, labels_batch in generator:
    start = time.time()
    features_batch = extractor.predict(inputs_batch)
    start = time.time()
    if (i + 1) * BATCH_SIZE > sample_count:
      features[i * BATCH_SIZE : sample_count , :] = features_batch
    else:
      features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE, : ] = features_batch
    i += 1
    if i * BATCH_SIZE >= sample_count:
      break
  
  return features


In [ ]:
import time
sketches_features = extract_features(conv_base, sketches_generator, 20000)
mirflickr_features = extract_features(conv_base, mirflickr_generator, 25000)

In [ ]:
print(sketches_features.shape)
print(mirflickr_features.shape)
print(sketches_features[0])
print()
print(mirflickr_features[0])
print(sketches_features[0].nbytes)
print(mirflickr_features[0].nbytes)

(20000, 2048)
(25000, 2048)
[0.68168402 0.25217029 0.22079533 ... 0.11126359 0.02828261 0.        ]

[0.95190179 0.05666274 0.78650057 ... 0.73523319 0.69135904 0.44517657]
16384
16384


In [ ]:
sketches_features[0,0]

0.6816840171813965

In [ ]:
lsh_base = LSH(feature_dim = sketches_features[0].shape[0], g=3, h=1)
lsh_base.insert(np.vstack((sketches_features, mirflickr_features)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features.shape[0]))))

In [ ]:
start = time.time()
query_result_ed = lsh_base.query(sketches_features[0], 80, return_cost = True)
end = time.time()
print(f'query time {end - start}')
query_result_sim = lsh_base.query(sketches_features[0], 80, mode='similarity', return_cost = True)

query time 0.13454461097717285


In [ ]:
print(query_result_ed)
print(query_result_sim)

({'ids': array(['airplane/1.png', 'airplane/49.png', 'submarine/16577.png',
       'submarine/16594.png', 'bread/2409.png', 'hamburger/7684.png',
       'fish/6624.png', 'space shuttle/15781.png',
       'rollerblades/13778.png', 'airplane/44.png', 'fish/6582.png',
       'airplane/2.png', 'couch/4657.png', 'floor lamp/6782.png',
       'flying bird/6909.png', 'flying saucer/7000.png', 'bee/1413.png',
       'canoe/3594.png', 'seagull/14704.png', 'computer-mouse/4589.png',
       'lightbulb/9718.png', 'teapot/17507.png', 'microscope/10431.png',
       'flashlight/6681.png', 'couch/4715.png', 'submarine/16597.png',
       'computer-mouse/4606.png', 'paper clip/11557.png',
       'standing bird/16315.png', 'trombone/18698.png',
       'mailbox/10110.png', 'bear (animal)/1218.png', 'purse/13234.png',
       'airplane/50.png', 'speed-boat/15859.png', 'sailboat/13998.png',
       'diamond/5222.png', 'helmet/8425.png', 'computer monitor/4510.png',
       'computer-mouse/4575.png', 'arm/472.p

In [ ]:
no_index_base = NO_INDEX()
no_index_base.insert(np.vstack((sketches_features, mirflickr_features)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features.shape[0]))))
start = time.time()
query_result_ed_no_index = no_index_base.query(sketches_features[0], 80, return_cost = True)
end = time.time()
print(f'query time {end - start}')
query_result_sim_no_index = no_index_base.query(sketches_features[0], 80, mode='similarity', return_cost = True)
print(query_result_ed_no_index)
print(query_result_sim_no_index)
print(query_result_sim_no_index[0]['ids'].shape)

query time 0.5169310569763184
({'ids': array(['airplane/1.png', 'airplane/49.png', 'airplane/12.png',
       'submarine/16577.png', 'submarine/16594.png', 'airplane/18.png',
       'hat/8073.png', 'bread/2409.png', 'satellite dish/14199.png',
       'mushroom/11110.png', 'hamburger/7684.png', 'tv/18979.png',
       'bread/2460.png', 'mailbox/10090.png', 'flying saucer/7002.png',
       'leaf/9633.png', 'mailbox/10087.png', 'axe/684.png',
       'sea turtle/14600.png', 'bulldozer/2628.png', 'teapot/17451.png',
       'axe/700.png', 'duck/5770.png', 'fish/6624.png',
       'space shuttle/15781.png', 'frog/7245.png',
       'rollerblades/13778.png', 'airplane/44.png', 'bell/1570.png',
       'seagull/14670.png', 'fish/6582.png', 'airplane/2.png',
       'couch/4657.png', 'seagull/14714.png', 'owl/11334.png',
       'floor lamp/6782.png', 'flying bird/6909.png',
       'hamburger/7725.png', 'flying saucer/7000.png', 'shoe/14977.png',
       'bus/2689.png', 'axe/683.png', 'paper clip/11548.

In [ ]:
lsh_bitwise = LSH(feature_dim = sketches_features[0].shape[0], g=3, h=1, bitwise_hash=True)
lsh_bitwise.insert(np.vstack((sketches_features, mirflickr_features)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features.shape[0]))))
start = time.time()
query_result_ed_bitwise = lsh_bitwise.query(sketches_features[0], 80, return_cost = True)
end = time.time()
print(f'query time {end - start}')
query_result_sim_bitwise = lsh_bitwise.query(sketches_features[0], 80, mode='similarity', return_cost = True)
print(query_result_ed_bitwise)
print(query_result_sim_bitwise)

query time 3.120476484298706
({'ids': array(['airplane/1.png', 'airplane/49.png', 'airplane/12.png',
       'submarine/16577.png', 'submarine/16594.png', 'airplane/18.png',
       'hat/8073.png', 'bread/2409.png', 'satellite dish/14199.png',
       'mushroom/11110.png', 'hamburger/7684.png', 'tv/18979.png',
       'bread/2460.png', 'mailbox/10090.png', 'flying saucer/7002.png',
       'leaf/9633.png', 'mailbox/10087.png', 'axe/684.png',
       'sea turtle/14600.png', 'bulldozer/2628.png', 'teapot/17451.png',
       'axe/700.png', 'duck/5770.png', 'fish/6624.png',
       'space shuttle/15781.png', 'frog/7245.png',
       'rollerblades/13778.png', 'airplane/44.png', 'bell/1570.png',
       'seagull/14670.png', 'fish/6582.png', 'airplane/2.png',
       'couch/4657.png', 'seagull/14714.png', 'owl/11334.png',
       'floor lamp/6782.png', 'flying bird/6909.png',
       'hamburger/7725.png', 'flying saucer/7000.png', 'shoe/14977.png',
       'bus/2689.png', 'axe/683.png', 'paper clip/11548.p

In [ ]:
def compare(a, b):
  for comp in zip(a, b):
    if comp[0] != comp[1]:
      print(comp)
      return False
  return True

In [ ]:
del lsh_bitwise

### Fine tuning with classification

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [ ]:
MODEL_PATH = "/content/gdrive/Shareddrives/COMPUTER_VISION/models"

In [ ]:
inception = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=INPUT_SHAPE)
inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    validation_split=0.1,
    zoom_range=0.1,
    rotation_range=180,
    vertical_flip = True,
    horizontal_flip = True
    ) # set validation split


train_generator = train_datagen.flow_from_directory(
    SKETCHES_DIR + '/train',
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    SKETCHES_DIR + '/train',
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    subset='validation') 

inception = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
# https://keras.io/api/applications/#finetune-inceptionv3-on-a-new-set-of-classes
x = inception.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(250, activation='softmax')(x)

model_classification = Model(inputs=inception.input, outputs=predictions)

for layer in inception.layers:
    layer.trainable = False

model_checkpoint = ModelCheckpoint('checkpoint.h5', save_best_only=True, save_weights_only=True, verbose=1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10)

model_classification.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model_classification.summary()

history = model_classification.fit(
  train_generator,
  steps_per_epoch = (20000 - 250 * test_images_per_class) * 0.9 // BATCH_SIZE,
  epochs=10,
  validation_data = validation_generator,
  validation_steps =  (20000 - 250 * test_images_per_class) * 0.1 // BATCH_SIZE,
  callbacks = [callback, model_checkpoint]
  )

model_classification.load_weights('checkpoint.h5')

# due inception block
for layer in model_classification.layers[:249]:
   layer.trainable = False
for layer in model_classification.layers[249:]:
   layer.trainable = True

from tensorflow.keras.optimizers import SGD
model_classification.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
model_classification.summary()
history = model_classification.fit(
  train_generator,
  steps_per_epoch = (20000 - 250 * test_images_per_class) * 0.9 // BATCH_SIZE,
  epochs=40,
  validation_data = validation_generator,
  validation_steps =  (20000 - 250 * test_images_per_class) * 0.1 // BATCH_SIZE,
  callbacks = [callback, model_checkpoint]
  )


model_classification.load_weights('checkpoint.h5')
models.save_model(model_classification, os.path.join(MODEL_PATH, 'inception_finetuning_classification.h5'))

Found 13500 images belonging to 250 classes.
Found 1500 images belonging to 250 classes.
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_1316 (Conv2D)            (None, None, None, 3 864         input_15[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1316 (Batch (None, None, None, 3 96          conv2d_1316[0][0]                
__________________________________________________________________________________________________
activation_1316 (Activation)    (None, None, None, 3 0           batch_normalization_1316[0][0]   
___

In [ ]:
loaded = models.load_model(os.path.join(MODEL_PATH, 'inception_finetuning_classification.h5'))
finetuned_extactor = Model(loaded.input, loaded.layers[-1].output)
finetuned_extactor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [ ]:
import time
sketches_features_finetuned = extract_features(finetuned_extactor, sketches_generator, 20000)
mirflickr_features_finetuned = extract_features(finetuned_extactor, mirflickr_generator, 25000)

In [ ]:
no_index_finetuned = NO_INDEX()
no_index_finetuned.insert(np.vstack((sketches_features_finetuned, mirflickr_features_finetuned)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features_finetuned.shape[0]))))

In [ ]:
query_result_ed = no_index_finetuned.query(sketches_features_finetuned[0], 80)
query_result_sim = no_index_finetuned.query(sketches_features_finetuned[0], 80, mode='similarity')

In [ ]:
print(sketches_features_finetuned.shape)
print(mirflickr_features_finetuned.shape)

(20000, 2048)
(25000, 2048)


In [ ]:
print(query_result_ed)
print(query_result_sim)

{'ids': array(['airplane/1.png', 'airplane/53.png', 'airplane/18.png',
       'airplane/2.png', 'airplane/50.png', 'airplane/12.png',
       'airplane/49.png', 'airplane/48.png', 'airplane/42.png',
       'airplane/44.png', 'flying saucer/7014.png',
       'flying bird/6909.png', 'airplane/70.png', 'airplane/54.png',
       'airplane/74.png', 'airplane/46.png', 'airplane/77.png',
       'flying saucer/7002.png', 'airplane/80.png', 'airplane/75.png',
       'airplane/40.png', 'airplane/56.png', 'tiger/17877.png',
       'airplane/21.png', 'space shuttle/15823.png', 'airplane/8.png',
       'airplane/37.png', 'penguin/12076.png', 'airplane/29.png',
       'airplane/58.png', 'submarine/16597.png', 'airplane/5.png',
       'flying saucer/7036.png', 'airplane/17.png', 'ship/14949.png',
       'cactus/2975.png', 'submarine/16604.png', 'flying saucer/6975.png',
       'submarine/16594.png', 'shark/14775.png',
       'space shuttle/15820.png', 'hamburger/7755.png', 'airplane/52.png',
       'a

### Evaluation

In [ ]:
def average_precision(requested_label, result_labels, n_ground_truth = 80):
  """
  label ricercata, label ottenute, il numero di oggetti che ci sono quella label
  """
  correct_array = (requested_label == result_labels).astype(int)
  precision_array = [np.mean(correct_array[:k]) for k in range(1, correct_array.shape[0] + 1)]
  # print(precision_array * correct_array) # mi rimangono solo quelli a 1
  return np.sum(precision_array * correct_array) / n_ground_truth


print(average_precision(1, np.array([1,0,1,1,1,1,0,0,0,1]), 6)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([0,1,0,0,1,1,1,0,1,0]), 5)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([1,0,1,1,1,1,0,0]), 6)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([0,1,0,0,1,1,1,0]), 5)) # importante cambiare l'ultimo parametro con il numero massimo di true label
print(average_precision(1, np.array([1,1,1,1, 0 ,0,0]), 4))

0.7749999999999999
0.5053968253968254
0.6749999999999999
0.3942857142857143
1.0


In [ ]:
query_result_ed = no_index_base.query(sketches_features[0], 80)
query_result_sim = no_index_base.query(sketches_features[0], 80, mode='similarity')
print('euclidean')
print(query_result_ed)
print(average_precision(0, query_result_ed['labels']))
print('similarity')
print(query_result_ed)
print(average_precision(0, query_result_ed['labels']))

euclidean
{'ids': array(['airplane/1.png', 'airplane/49.png', 'airplane/12.png',
       'submarine/16577.png', 'submarine/16594.png', 'airplane/18.png',
       'hat/8073.png', 'bread/2409.png', 'satellite dish/14199.png',
       'mushroom/11110.png', 'hamburger/7684.png', 'tv/18979.png',
       'bread/2460.png', 'mailbox/10090.png', 'flying saucer/7002.png',
       'leaf/9633.png', 'mailbox/10087.png', 'axe/684.png',
       'sea turtle/14600.png', 'bulldozer/2628.png', 'teapot/17451.png',
       'axe/700.png', 'duck/5770.png', 'fish/6624.png',
       'space shuttle/15781.png', 'frog/7245.png',
       'rollerblades/13778.png', 'airplane/44.png', 'bell/1570.png',
       'seagull/14670.png', 'fish/6582.png', 'airplane/2.png',
       'couch/4657.png', 'seagull/14714.png', 'owl/11334.png',
       'floor lamp/6782.png', 'flying bird/6909.png',
       'hamburger/7725.png', 'flying saucer/7000.png', 'shoe/14977.png',
       'bus/2689.png', 'axe/683.png', 'paper clip/11548.png',
       'blimp/1

In [ ]:
from random import random
def mAP(index, features, n_queries = 160, n_labels = 250, img_per_labels = 80, mode = 'euclidean'):
  sum = 0
  for i in range(n_queries):
    label = i % n_labels
    image_idx = ((i * img_per_labels) + int(random() * img_per_labels)) % (n_labels * img_per_labels)
    # print('QUERY')
    # print('index = ' + str(image_idx))
    # print('label =' + str(label))
    res = index.query(features[image_idx], img_per_labels, mode = mode)
    # print('first label of resultset (must be equal to label) = ' + str(res['labels'][0]))
    assert res['labels'][0] == label, 'deve essere della stessa label'
    sum += average_precision(label, res['labels'], img_per_labels)
  return sum / n_queries


In [ ]:
map_value = mAP(no_index_base, sketches_features)
print(map_value)

0.07359088158586594


In [ ]:
map_value = mAP(no_index_base, sketches_features, mode='similarity')
print(map_value)

0.07483105657810528


In [ ]:
map_value = mAP(no_index_finetuned, sketches_features_finetuned)
print(map_value)

0.15644114465575926


In [ ]:
map_value = mAP(no_index_finetuned, sketches_features_finetuned, mode='similarity')
print(map_value)

0.1712772914825181


In [ ]:
# try to calculate bucket dispersion
# ritorna la percentuale media, la deviazione standard della classe più popolosa dei bucket,
# ritorna anche il numero di bucket
def bucket_dispersion(index):
  percs = None
  itemsCount = None
  i = 0
  for bucket in index._index:
    i += 1
    _, counts = np.unique(index._index[bucket]['labels'], return_counts=True)
    perc = np.max(counts) / np.sum(counts)
    itemCount = np.sum(counts)
    if percs is None:
      percs = np.array([perc])
      itemsCount = np.array([itemCount])
    else:
      percs = np.concatenate((percs, np.array([perc])))
      itemsCount = np.concatenate((itemsCount, np.array([itemCount])))
  return {
      'perc': {
          'mean': np.mean(percs),
          'deviation': np.std(percs)
      },
      'count': {
          'average': np.mean(itemsCount),
          'deviation': np.std(itemsCount)
      },
      'n_buckets': i,
      'n_items_counting_duplicates': np.sum(itemsCount) # ci sono anche i duplicati
  }
print(bucket_dispersion(lsh_base))
print(bucket_dispersion(lsh_finetuned))
